# Declarative data analysis

This notebook demonstrates a "realistic" set of operations and pipeline you might write using functions from `utilz` combined with the `toolz` package. It mimics the declarative style of functional-programming more recently popularized in `R` libraries like `dplyr`. 

In [1]:
import pandas as pd
from utilz import randdf, assign, rename
from toolz import pipe

# Verb-based dataframe manipulation

One of the strengths of libraries like `dplyr` is that it provides a sort of *grammar* for manipulating data. This allows for concise analysis pipelines. `utilz` offers some similar functionality through a few choice *verbs*. Tons of additional functionality is available in the great [plydata](https://plydata.readthedocs.io/en/stable/) library:

In [18]:
# Setup some random data
df = pipe(
    randdf((20, 3)),
    assign(D1=list("abcde") * 4),
    rename({"A1": "rt", "B1": "score", "C1": "speed", "D1": "group"}),
    assign(rt_doubled="rt*2"),
)
df.head()

,rt,score,speed,group,rt_doubled
0,0.053000,0.943329,0.687069,a,0.106001
1,0.655014,0.351922,0.460340,b,1.310027
2,0.458604,0.775281,0.956761,c,0.917208
3,0.108500,0.892330,0.051870,d,0.217001
4,0.916564,0.937654,0.825559,e,1.833127


## Basic slicing/subsetting
`rows` and `cols` behave similarly and can index a Dataframe using *strings*, *lists/arrays*, *tuples* or *ints*:

In [4]:
from utilz import rows, cols

### strings

`rows` interpret strings as `df.query` expressions, while `cols` looks for a column name:

In [21]:
pipe(df, 
    rows("group == 'c' or group == 'b'"), 
    cols("rt")
    )


,rt
0,0.655014
1,0.458604
2,0.913827
3,0.411440
4,0.798876
5,0.902748
6,0.180981
7,0.090812


Cols can also "negatively index" columns which selects everything *except* the column:

In [22]:
pipe(df, 
    rows("group == 'c' or group == 'b'"), 
    cols("-rt")
    )


,score,speed,group,rt_doubled
0,0.351922,0.460340,b,1.310027
1,0.775281,0.956761,c,0.917208
2,0.022340,0.887066,b,1.827654
3,0.190052,0.928055,c,0.822879
4,0.022913,0.459544,b,1.597753
5,0.025795,0.486684,c,1.805496
6,0.287260,0.784425,b,0.361963
7,0.099929,0.708791,c,0.181623


### ints

Both `rows` and `cols` interpret integers as *numerical indices* (i.e. `.iloc`):

In [23]:
# Second row, third column
pipe(df,
    rows(1),
    cols(2)
     )

,speed
1,0.46034


### lists/arrays

Both `rows` and `colws` interprets lists and arrays as *numerical indices* (i.e. `.iloc`) if they contain numbers, but as *row/col labels* (i.e. `.loc`) if they contain strings.

In [7]:
pipe(df, 
    rows([0, 2, 4]), 
    cols(["rt", "speed", "score"])
    )


,rt,speed,score
0,0.600285,0.198946,0.696828
2,0.277055,0.809752,0.983424
4,0.062525,0.500357,0.764594


In [8]:
# Same but using indexes for columns
pipe(df, 
    rows([0, 2, 4]), 
    cols([0, 2, 1])
    )


,rt,speed,score
0,0.600285,0.198946,0.696828
2,0.277055,0.809752,0.983424
4,0.062525,0.500357,0.764594


### tuples

Both `rows` and `cols` interpret tuples as `(start, stop, step)` slices, with `step` being optional:

In [9]:
pipe(df, 
     rows((1, 11, 2)), 
     cols((0, 5, 2)))


,rt,speed,rt_doubled
1,0.568678,0.836273,1.137357
3,0.193848,0.026424,0.387695
5,0.509818,0.099443,1.019636
7,0.987609,0.434809,1.975218
9,0.022993,0.565018,0.045986


In [10]:
pipe(df,
     rows((0, 5))
)

,rt,score,speed,group,rt_doubled
0,0.600285,0.696828,0.198946,a,1.200570
1,0.568678,0.436753,0.836273,b,1.137357
2,0.277055,0.983424,0.809752,c,0.554110
3,0.193848,0.632703,0.026424,d,0.387695
4,0.062525,0.764594,0.500357,e,0.125049


## Summarizing data
To perform an operation that results in a *scalar* output per column, use `summarize`. It always return a *smaller* dataframe or series than the original

Non-grouped inputs produce *Series* results:

In [5]:
from utilz import summarize

pipe(df, 
    rows("group == 'c' or group == 'b'"), 
    summarize(rt='mean',speed='mean')
    )

rt       0.435085
speed    0.474883
dtype: float64

Grouped inputs product *DataFrame* results:

In [6]:
from utilz import groupby

pipe(df, 
    groupby('group'), 
    summarize(score = 'mean', rt = 'std')
    )

,score,rt
group,,
a,0.660371,0.382023
b,0.541034,0.276673
c,0.436452,0.280261
d,0.443445,0.442302
e,0.395343,0.267489


## Transforming data
To perfom an operation that results in a *non-scalar* output per column, or to ensure that an operation with *scalar* output retains the original shape of the input, use `assign`. It always returns back a DataFrame the *same* size as the original via broadcasting.

With non-grouped inputs, the mean score is broadcasted across the *entire* DataFrame

In [7]:
pipe(df, 
     assign(mean_overall_score="score.mean()")
     )


,rt,score,speed,group,rt_doubled,mean_overall_score
0,0.369143,0.957854,0.401780,a,0.738285,0.495329
1,0.375460,0.946958,0.144849,b,0.750919,0.495329
2,0.471786,0.178436,0.928845,c,0.943571,0.495329
3,0.572397,0.458051,0.811822,d,1.144795,0.495329
4,0.146566,0.113969,0.691174,e,0.293131,0.495329
5,0.685974,0.541836,0.641309,a,1.371949,0.495329
6,0.787281,0.313961,0.344903,b,1.574563,0.495329
7,0.133493,0.396424,0.654379,c,0.266987,0.495329
8,0.020939,0.306361,0.392558,d,0.041877,0.495329
9,0.501037,0.812635,0.603648,e,1.002075,0.495329


With grouped inputs, the mean score is broadcasted *within* each group:

In [8]:
pipe(df, 
    groupby('group'), 
    assign(mean_grp_score = 'score.mean()')
    )

,rt,score,speed,group,rt_doubled,mean_grp_score
0,0.369143,0.957854,0.401780,a,0.738285,0.660371
1,0.375460,0.946958,0.144849,b,0.750919,0.541034
2,0.471786,0.178436,0.928845,c,0.943571,0.436452
3,0.572397,0.458051,0.811822,d,1.144795,0.443445
4,0.146566,0.113969,0.691174,e,0.293131,0.395343
5,0.685974,0.541836,0.641309,a,1.371949,0.660371
6,0.787281,0.313961,0.344903,b,1.574563,0.541034
7,0.133493,0.396424,0.654379,c,0.266987,0.436452
8,0.020939,0.306361,0.392558,d,0.041877,0.443445
9,0.501037,0.812635,0.603648,e,1.002075,0.395343


Here no broadcasting occurs, but the result is reshaped while respecting groups like `transform` in pandas.

In [9]:
pipe(df, 
    groupby('group'), 
    assign(
        score_centered='score - score.mean()', 
        score_norm = 'score/score.std()',
        score_zscore ='(score - score.mean()) / score.std()'
        )
        
    )

,rt,score,speed,group,rt_doubled,score_centered,score_norm,score_zscore
0,0.369143,0.957854,0.401780,a,0.738285,0.297483,3.900577,1.211410
1,0.375460,0.946958,0.144849,b,0.750919,0.405924,3.195080,1.369608
2,0.471786,0.178436,0.928845,c,0.943571,-0.258016,0.650171,-0.940141
3,0.572397,0.458051,0.811822,d,1.144795,0.014607,1.517082,0.048378
4,0.146566,0.113969,0.691174,e,0.293131,-0.281374,0.381001,-0.940639
5,0.685974,0.541836,0.641309,a,1.371949,-0.118536,2.206465,-0.482701
6,0.787281,0.313961,0.344903,b,1.574563,-0.227072,1.059320,-0.766152
7,0.133493,0.396424,0.654379,c,0.266987,-0.040028,1.444460,-0.145852
8,0.020939,0.306361,0.392558,d,0.041877,-0.137083,1.014679,-0.454024
9,0.501037,0.812635,0.603648,e,1.002075,0.417292,2.716654,1.395013


Assignments can be arbitrary complex as long as they can be expressed as a string. They can even make reference to other columns:

In [10]:
pipe(df,
    groupby('group'),
     assign(
         price="(rt > .5) * speed"
     )
)

,rt,score,speed,group,rt_doubled,price
0,0.369143,0.957854,0.401780,a,0.738285,0.000000
1,0.375460,0.946958,0.144849,b,0.750919,0.000000
2,0.471786,0.178436,0.928845,c,0.943571,0.000000
3,0.572397,0.458051,0.811822,d,1.144795,0.811822
4,0.146566,0.113969,0.691174,e,0.293131,0.000000
5,0.685974,0.541836,0.641309,a,1.371949,0.641309
6,0.787281,0.313961,0.344903,b,1.574563,0.344903
7,0.133493,0.396424,0.654379,c,0.266987,0.000000
8,0.020939,0.306361,0.392558,d,0.041877,0.000000
9,0.501037,0.812635,0.603648,e,1.002075,0.603648


You can also combine them with some handy helpers from `plydata`, but you'll have to pass the `use_ply=True` if you're using `groupby`:

In [11]:
from plydata import case_when

df = pipe(
    df,
    groupby("group", use_ply=True),
    assign(
        mean_grp_speed="speed.mean()",
        speed_tier=case_when(
            {
                'speed < speed.mean()': '"one"',
                'speed == speed.mean()': '"two"',
                'speed > speed.mean()': '"three"',
            }
        )
    ),
)
df

groups: ['group']
          rt     score     speed group  rt_doubled  mean_grp_speed speed_tier
0   0.369143  0.957854  0.401780     a    0.738285        0.628344        one
1   0.375460  0.946958  0.144849     b    0.750919        0.198228        one
2   0.471786  0.178436  0.928845     c    0.943571        0.751539      three
3   0.572397  0.458051  0.811822     d    1.144795        0.695395      three
4   0.146566  0.113969  0.691174     e    0.293131        0.532096      three
5   0.685974  0.541836  0.641309     a    1.371949        0.628344      three
6   0.787281  0.313961  0.344903     b    1.574563        0.198228      three
7   0.133493  0.396424  0.654379     c    0.266987        0.751539        one
8   0.020939  0.306361  0.392558     d    0.041877        0.695395        one
9   0.501037  0.812635  0.603648     e    1.002075        0.532096      three
10  0.884708  0.394330  0.761686     a    1.769416        0.628344      three
11  0.254019  0.576493  0.094998     b    0.508038        0.198228        one
12  0.634474  0.823560  0.913038     c    1.268948        0.751539      three
13  0.011011  0.153273  0.932635     d    0.022021        0.695395      three
14  0.800533  0.273083  0.373999     e    1.601066        0.532096        one
15  0.011422  0.747465  0.708599     a    0.022844        0.628344      three
16  0.785102  0.326722  0.208163     b    1.570203        0.198228      three
17  0.039069  0.347389  0.509891     c    0.078137        0.751539        one
18  0.913666  0.856093  0.644563     d    1.827332        0.695395        one
19  0.503072  0.381687  0.459562     e    1.006144        0.532096        one

In [25]:
pipe(df, 
     assign(
         relabel=case_when(
             {
                 "speed_tier == 'one'": "'winner'",
                 "speed_tier == 'two'": "'loser'",
                 "speed_tier == 'three'": "'loser'",
                 }
             )
         )
     )


groups: ['group']
          rt     score     speed group  rt_doubled  mean_grp_speed speed_tier  \
0   0.835214  0.469788  0.818336     a    1.670428        0.689456      three   
1   0.933498  0.803162  0.836482     b    1.866995        0.741970      three   
2   0.870527  0.988006  0.045092     c    1.741054        0.342046        one   
3   0.331299  0.188222  0.201379     d    0.662598        0.507731        one   
4   0.060063  0.527094  0.873889     e    0.120126        0.613322      three   
5   0.663402  0.772564  0.730931     a    1.326803        0.689456      three   
6   0.881181  0.562835  0.373125     b    1.762361        0.741970        one   
7   0.772267  0.104532  0.375590     c    1.544535        0.342046      three   
8   0.145882  0.500106  0.343859     d    0.291764        0.507731        one   
9   0.834803  0.905774  0.474066     e    1.669605        0.613322        one   
10  0.697991  0.060713  0.765796     a    1.395981        0.689456      three   
11  0.602443  0.370808  0.787415     b    1.204887        0.741970      three   
12  0.663916  0.298299  0.302135     c    1.327832        0.342046        one   
13  0.762938  0.904471  0.933878     d    1.525876        0.507731      three   
14  0.203782  0.463020  0.827351     e    0.407565        0.613322      three   
15  0.371209  0.715934  0.442762     a    0.742419        0.689456        one   
16  0.799226  0.782305  0.970858     b    1.598452        0.741970      three   
17  0.043911  0.105998  0.645368     c    0.087822        0.342046      three   
18  0.898024  0.307063  0.551807     d    1.796048        0.507731      three   
19  0.855099  0.097262  0.277984     e    1.710198        0.613322        one   

   relabel  
0    loser  
1    loser  
2   winner  
3   winner  
4    loser  
5    loser  
6   winner  
7    loser  
8   winner  
9   winner  
10   loser  
11   loser  
12  winner  
13   loser  
14   loser  
15  winner  
16   loser  
17   loser  
18   loser  
19  winner

# Efficient analyses using memoization, currying, and caching

It's convenient to combine these operations using `pipe`, but what if some operations take longer than others? It would be annoying to have to rerun expensive i/o or computate operations while you're debugging or adding another step to your pipeline. This section demonstrates three ways that can dramatically improve efficiency.

## Memoization

**Memoize** a function to save its last input in memory (RAM) and recall it when called with the same arguments rather than re-executing a potentially long running function. Memoized outputs *do not* persist across kernel restarts, and aren't great for functions that have very large or non-pickleable inputs and outputs. A nice use case is simply loading a file. Given the filepath, you're loading the same file each time, so lets just memoize the result, especially if it takes a long time to reread the file:

In [16]:
from toolz import memoize
from time import sleep

@memoize
def load(path):
    "Simulate slow loading a file..."
    print("loading from disk")
    sleep(5)
    return pd.read_csv(path)

## Currying
**Curry**, also called *partial function application*, is helpful when writing custom functions you want to add to a `pipe`. When curried, a function operates normally when it receives all its required arguments, but turns into a *partial* function when it gets fewer than all its required arguments. This partial function behaves just like the original except with a subset of its arguments "fixed". Because `pipe` implicitly passes the output of the last function to the next function in the pipeline, it's not possible to manipulate secondary arguments to a function in the pipeline without currying.


When writing curried functions for use with `pipe`, make sure the expected output from the previous function execution in a pipeline is the *last* `arg` the function receives. Any required `args` should appear *before* this value and any `kwargs` should appear after:

In [6]:
from toolz import curry

# Curried function args and kwargs should have the form:
# required args, dataframe as the last required arg, optional kwargs

# No optional args so required args *before* df which is passed by pipe
@curry
def calc_mean_score(nrows, df):
    return df.loc[:nrows, 'score'].mean()

pipe(df,
     calc_mean_score(5)
    )

0.41179621192888827

In [7]:
# Optional kwarg which comes *after* df which is passed by pipe
@curry
def calc_mean_score(nrows, df, normalize=False):
    out = df.loc[:nrows, 'score'].mean()
    if normalize:
        out /= df.loc[:nrows, 'score'].std()
    return out

pipe(df, 
     calc_mean_score(5, normalize=True)
     )

1.8490069051307567

## Caching outputs to disk
**Cache** so the result of a function is stored to disk in a file made unique by hash of the args and kwargs to the function. Use `utilz.disk_cache` to decorate a function so it caches, which works similar to `toolz.memoize` but stores the result to a file (rather than in memory) and loads the file when called with the same inputs. Essentially you're trading computation time for i/o time. This is usually preferable to `memoize` for larger memory hungry inputs and outputs, and necessary if input or outputs cannot be pickled (e.g. dataframes, arrays, deep objects, etc). Setting the threshold to something like 1 essentially always caches the result. 

In [19]:
from utilz import disk_cache

# We're also using the curry decorator so norm works in the pipeline below
# It's not needed to use the disk_cache decorator

@curry
@disk_cache(threshold=1)
def norm(df, num='', denom=''):
    "Simulate expensive function that takes args"
    print("computing...")
    sleep(5)
    return pd.DataFrame({"norm": df[num] / df[denom]})

# Putting it together: Pipeline based data analysis with caching and memoization

First you might want to clear any local cache, i.e. the `.utilz_cache` folder. Not neccessary, but important so you don't accidently load a disk-cached output without realizing. `utilz` will always let you know if it's caching or loading a cached result to help you.


In [21]:
from utilz import clear_cache

# Clear any existing cache
clear_cache()

Because we used the `memoize` decorator when defining `load` above, only the first run of this pipeline actually loads the data and incurs an i/o cost. Likewise because `norm` is decorated with `disk_cache` only the first run of norm (with the same prior pipeline outputs) incurs a compute cost.

In [22]:
summary = pipe(
    "test.csv",
    load,
    groupby("group"),
    assign(
        score_centered="score - score.mean()", 
        score_mean="score.mean()", 
        score_std="score.std()"),
    norm(num='rt',denom='score')
)
summary

loading from disk
computing...
Exceeded compute time. Result cached to .utilz_cache/norm___denom__score--df__9c405c0813bd657babdcd3583202ebea2a7a61819abdb234b93c59dab0d05fcc--num__rt.csv


,norm
0,0.876757
4,0.219101
8,1.803562
12,0.080123
16,1.397509
1,0.013055
5,0.293790
9,0.678440
13,1.044078
17,0.495721


Notice how no data loading or expensive computation needs to happen on a second run of this pipeline (no print messages aside from `utilz` telling you it's loading `norm` last cached result):

In [24]:
summary = pipe(
    "test.csv",
    load,
    groupby("group"),
    assign(
        score_centered="score - score.mean()", 
        score_mean="score.mean()", 
        score_std="score.std()"),
    norm(num='rt',denom='score')
)
summary

Returning norm cached result


,norm
0,0.876757
1,0.219101
2,1.803562
3,0.080123
4,1.397509
5,0.013055
6,0.293790
7,0.678440
8,1.044078
9,0.495721


# Summary


This setup is nice because it allows for both interactive data analysis as well as reproducible scripts. Simply start writing the pipeline steps, and comment out ones you want to skip or debug. In another notebook cell edit the source code of a function in the pipeline and incrementally add to its body, while rerunning the pipeline to see results as you build up your functions.

For functions that take a while to run, try decorating them with `memoize` or `disk_cache`. Memoize is nice for loading csv/text files (so you don't need to re-read them from disk each re-run of the pipeline). Cacheing is nice for expensive operations or operations on complex datastructures like arrays and dataframes. Plus, utilz saves them in standard robust file types (.csv. or .h5 rather than pickles) so you're also getting incremental backups of your work. No more need to rely on saved "state" in a Juptyer notebook.